<a href="https://colab.research.google.com/github/GustavoMarques22/ProjetoAlura-AreaDaSaude/blob/main/ProjetoALURA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Criação de Conteúdo Inteligente para Conexão Saúde-IA

## Unindo Conhecimento e Acessibilidade na Área da Saúde

A comunicação eficaz na saúde é um pilar para o bem-estar. No entanto, tanto pacientes quanto profissionais enfrentam o desafio de encontrar, compreender e compartilhar informações médicas precisas e atualizadas em meio a um volume crescente de dados. Pacientes buscam clareza e confiança em suas jornadas de saúde, enquanto profissionais precisam de acesso rápido a pesquisas e avanços para aprimorar suas práticas.

## Nossa Proposta: Um Sistema de Agentes de IA como Ponte de Conhecimento

Este projeto apresenta um sistema inovador, impulsionado por Inteligência Artificial e arquitetura de agentes, projetado para automatizar e otimizar a criação de conteúdo de saúde. Nossa solução visa construir uma ponte de conhecimento, transformando informações complexas em conteúdo acessível e relevante para pacientes e profissionais da saúde, promovendo uma conexão mais forte e informada.

## O Impacto da IA na Comunicação em Saúde

Nosso objetivo é demonstrar como agentes de IA podem atuar de forma colaborativa para:

*   **Identificar Tendências:** Descobrir os avanços mais recentes e relevantes na área da saúde.
*   **Planejar Conteúdo Estratégico:** Organizar informações e definir abordagens que ressoem com diferentes públicos.
*   **Criar Conteúdo Engajador:** Redigir materiais claros, precisos e adaptados para plataformas como o Instagram.
*   **Garantir a Qualidade e Precisão:** Revisar o conteúdo para assegurar sua confiabilidade e adequação.

Através deste projeto, buscamos explorar o potencial da IA para transformar a maneira como a informação de saúde é criada e compartilhada, tornando-a mais dinâmica, acessível e impactante.

In [16]:
# Instalação das bibliotecas necessárias para o projeto de Saúde-IA
# Google GenAI para interagir com os modelos Gemini
# Google ADK para construir a arquitetura de agentes

%pip -q install google-genai google-adk

In [35]:
# Configuração da API Key do Google AI Studio
# A API Key deve ser armazenada no Colab Secrets com o nome 'GOOGLE_API_KEY'
import os
from google.colab import userdata

try:
    # Tenta obter a API Key dos segredos do Colab
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    if not os.environ["GOOGLE_API_KEY"]:
        # Se a API Key não for encontrada ou estiver vazia nos segredos
        raise ValueError("API Key não encontrada nos segredos do Colab.")
    print("✅ API Key configurada com sucesso a partir dos segredos do Colab.")
except Exception as e:
    # Captura e exibe o erro se a API Key não puder ser configurada
    print(f"❌ Erro ao configurar a API Key: {e}")
    print("Por favor, adicione sua GOOGLE_API_KEY nos segredos do Colab (ícone de chave 🔑 na barra lateral esquerda).")

# Inicializa o cliente Gemini com o modelo padrão
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash" # Modelo recomendado para este projeto
print(f"✨ Cliente Gemini inicializado com o modelo {MODEL_ID}")

# Opcional: Verificar modelos disponíveis (descomente se necessário)
# for m in client.models.list():
#   if 'generateContent' in m.supported_generation_methods:
#     print(m.name)

✅ API Key configurada com sucesso a partir dos segredos do Colab.
✨ Cliente Gemini inicializado com o modelo gemini-2.0-flash


## Ferramentas Essenciais e Funções Auxiliares

Para construir o nosso sistema de agentes, contamos com o **Google ADK (Agent Development Kit)**, que simplifica a criação e orquestração de agentes inteligentes, e a **ferramenta de busca do Google**, fundamental para coletar informações atualizadas. Além disso, desenvolvemos funções auxiliares para facilitar a interação com os agentes e a apresentação dos resultados.

In [18]:
# Importações e Funções Auxiliares para o ADK e interação
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search # Ferramenta de busca do Google
from google.genai import types # Para criar conteúdos (Content e Part)
from datetime import date # Para obter a data atual
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, HTML # Para exibir texto formatado no Colab
import requests # Pode ser útil para futuras integrações (ex: APIs de saúde)
import warnings # Para gerenciar warnings

# Ignorar warnings para uma saída mais limpa durante a execução
warnings.filterwarnings("ignore")

# --- Função auxiliar: Chamar um Agente via Runner ---
def call_agent(agent: Agent, message_text: str) -> str:
    """
    Envia uma mensagem de texto para um agente usando o Runner do ADK
    e retorna a resposta final consolidada.
    """
    # Utiliza o serviço de sessão em memória para simplificar
    session_service = InMemorySessionService()
    # Cria ou recupera uma sessão para o usuário e aplicativo
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Inicializa o Runner para gerenciar a execução do agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Prepara o conteúdo da mensagem para o agente
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Executa o agente e itera sobre os eventos até obter a resposta final
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# --- Função auxiliar: Formatar texto para Markdown no Colab ---
def to_markdown(text):
  """
  Formata texto para exibição em Markdown no Colab, ajustando a indentação
  e substituindo marcadores comuns por bullet points Markdown.
  """
  text = text.replace('•', '  *') # Substitui bullet points comuns por formato Markdown
  # Indenta o texto para destacar a saída dos agentes
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## A Arquitetura de Agentes para Criação de Conteúdo em Saúde

Nosso sistema de criação de conteúdo para conexão Saúde-IA é estruturado como um pipeline de agentes especializados. Cada agente tem uma função específica e colabora com os demais, passando a informação adiante para refinar e construir o post final. Essa abordagem modular permite flexibilidade e clareza no processo.

A cadeia de agentes funciona da seguinte forma:

1.  **Agente Buscador de Tendências em Saúde:** Inicia o processo buscando informações relevantes.
2.  **Agente Planejador de Conteúdo para Conexão Saúde:** Recebe as informações buscadas e estrutura o plano do post, pensando nos públicos.
3.  **Agente Redator de Posts Engajadores de Saúde:** Utiliza o plano para escrever o rascunho do post, com foco na linguagem e tom adequados.
4.  **Agente Revisor de Qualidade em Saúde:** Avalia e aprimora o rascunho, garantindo precisão e adequação.

Esta arquitetura sequencial garante que cada etapa da criação de conteúdo seja tratada por um especialista em IA, resultando em um post mais preciso, relevante e engajador.

## Agente 1: Buscador de Tendências em Saúde

Este agente é o ponto de partida do nosso pipeline. Sua função é essencial para garantir que o conteúdo gerado seja relevante e baseado nas informações mais recentes na área da saúde. Ele utiliza a ferramenta de busca do Google para identificar pesquisas, avanços tecnológicos e lançamentos importantes.

Para otimizar a busca, as instruções do agente o direcionam a:

*   Buscar informações **atuais**, focando nos últimos meses.
*   Priorizar **fontes confiáveis** na área da saúde (órgãos oficiais, instituições de pesquisa, hospitais de referência).
*   Identificar **lançamentos ou tendências com impacto significativo** para pacientes e profissionais.
*   Limitar a quantidade de resultados para manter o foco.

In [36]:
# Definição do Agente Buscador de Tendências em Saúde
def agente_buscador_saude(topico_saude: str, data_de_hoje: str) -> str:
    """
    Agente que busca informações relevantes sobre tendências e notícias na área da saúde,
    focando em avanços, pesquisas e lançamentos importantes de fontes confiáveis.

    Args:
        topico_saude: O tópico de saúde a ser buscado.
        data_de_hoje: A data atual no formato dd/mm/yyyy.

    Returns:
        Uma string contendo as informações encontradas sobre tendências e lançamentos.
    """
    buscador = Agent(
        name="agente_buscador_saude",
        model=MODEL_ID, # Usando o modelo definido no setup inicial
        instruction=f"""
        Você é um assistente de pesquisa especializado em notícias e tendências na área da saúde.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar as informações mais recentes e relevantes
        sobre avanços médicos, tecnológicos, novas pesquisas e lançamentos importantes relacionados ao tópico de saúde: "{topico_saude}".

        Foque em identificar no máximo 5 lançamentos ou tendências que demonstrem um impacto significativo para profissionais
        e/ou pacientes, com base na quantidade e relevância das notícias sobre eles.
        Priorize informações de fontes confiáveis como hospitais de referência, universidades, revistas científicas (Ex: The Lancet, NEJM),
        órgãos de saúde (Ex: OMS, Ministério da Saúde) e sites de notícias de saúde renomados.

        Os resultados devem ser atuais, de no máximo um mês antes da data de hoje ({data_de_hoje}).
        Seja conciso e apresente os resultados de forma clara.

        Formato da saída: Liste cada lançamento/tendência encontrado em um item separado,
        com um breve resumo do que se trata e a fonte principal (nome do site ou organização) entre parênteses, se possível.
        """,
        description="Agente que busca informações no Google sobre tendências na área da saúde",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico de saúde para buscar tendências e lançamentos: {topico_saude}\nData de hoje: {data_de_hoje}"
    informacoes_saude = call_agent(buscador, entrada_do_agente_buscador)
    return informacoes_saude

### Exemplo de Execução do Agente Buscador de Tendências em Saúde

Vamos ver o Agente Buscador em ação. Ele solicitará um tópico de saúde e então utilizará a ferramenta de busca para encontrar as tendências e lançamentos mais relevantes, exibindo o resultado.

In [37]:
# Exemplo de execução do Agente Buscador de Tendências em Saúde
data_de_hoje = date.today().strftime("%d/%m/%Y")

# --- Obter o Tópico de Saúde do Usuário ---
topico_saude_exemplo = input("❓ Por favor, digite o TÓPICO de saúde para buscar tendências: ")

if not topico_saude_exemplo:
    print("Você esqueceu de digitar o tópico de saúde!")
else:
    print(f"\n🔎 Buscando informações sobre tendências em {topico_saude_exemplo} na área da saúde...")

    # Chama o agente buscador de saúde
    informacoes_buscadas = agente_buscador_saude(topico_saude_exemplo, data_de_hoje)

    print("\n--- 📝 Resultado do Agente 1 (Buscador de Tendências em Saúde) ---\n")
    display(to_markdown(informacoes_buscadas))
    print("--------------------------------------------------------------")

# Para os próximos exemplos, armazenamos o tópico e as informações buscadas
# para que os agentes subsequentes possam utilizá-los.
if 'informacoes_buscadas' in locals() and informacoes_buscadas:
    # Variáveis para serem usadas pelos próximos exemplos
    topico_para_planejar = topico_saude_exemplo
    informacoes_para_planejar = informacoes_buscadas
    print("\n💾 Informações buscadas armazenadas para o próximo agente.")
else:
     print("\n❌ Nenhuma informação buscada. Os próximos exemplos podem não funcionar corretamente sem a execução bem-sucedida do Agente Buscador.")

❓ Por favor, digite o TÓPICO de saúde para buscar tendências: Saúde Mental

🔎 Buscando informações sobre tendências em Saúde Mental na área da saúde...

--- 📝 Resultado do Agente 1 (Buscador de Tendências em Saúde) ---



> Aqui estão algumas tendências e lançamentos recentes na área de Saúde Mental, com foco nos últimos meses:
> 
> 
> Com base nas informações mais recentes, aqui estão algumas tendências e lançamentos importantes na área de Saúde Mental:
> 
> 1.  **Avanços na Telemedicina:** A telemedicina continua a se expandir, oferecendo terapia online e consultas virtuais de saúde mental. A tecnologia está tornando o suporte à saúde mental mais acessível, especialmente para aqueles que não têm acesso à terapia tradicional (Continental Hospitals, Faculdade de Medicina Santa Marcelina).
> 2.  **IA no Tratamento da Saúde Mental:** A inteligência artificial está sendo usada para prevenir suicídios, tratar dependências químicas e oferecer suporte psicológico personalizado. Algoritmos avançados monitoram sintomas de ansiedade e depressão em tempo real (Plusoft).
> 3.  **Realidade Virtual (VR) na Saúde Mental:** A realidade virtual está se tornando uma ferramenta poderosa no tratamento de questões de saúde mental, permitindo simular situações desafiadoras e ajudar os pacientes a enfrentar seus medos e traumas de forma controlada (Laboratório Goes).
> 4.  **Wearables e Monitoramento Emocional:** Dispositivos vestíveis, como smartwatches, estão sendo integrados com aplicativos de saúde mental para monitorar sinais vitais e oferecer feedback em tempo real sobre o estado emocional dos usuários (4Medic, Laboratório Goes).
> 5.  **Programas de Apoio Psicológico nas Empresas:** Empresas estão implementando programas de atendimento psicológico remoto e presencial, jornadas de trabalho flexíveis e treinamentos para líderes sobre empatia e gestão de conflitos (SER HCM).


--------------------------------------------------------------

💾 Informações buscadas armazenadas para o próximo agente.


## Agente 2: Planejador de Conteúdo para Conexão Saúde

Com as informações relevantes encontradas pelo Agente Buscador, o Agente Planejador entra em ação. Sua responsabilidade é analisar essas informações e estruturar um plano detalhado para a criação de um post que ressoe com o público de saúde, tanto pacientes quanto profissionais. Ele utiliza a ferramenta de busca para aprofundar nos temas e identificar os pontos mais relevantes e o impacto prático das tendências e lançamentos.

In [38]:
# Definição do Agente Planejador de Conteúdo para Conexão Saúde
def agente_planejador_saude(topico_saude: str, informacoes_buscadas: str) -> str:
    """
    Agente que planeja o conteúdo de um post de saúde, analisando informações buscadas
    e identificando pontos relevantes para pacientes e profissionais.

    Args:
        topico_saude: O tópico de saúde principal do conteúdo.
        informacoes_buscadas: As informações encontradas pelo Agente Buscador.

    Returns:
        Uma string contendo o plano detalhado para o post.
    """
    planejador = Agent(
        name="agente_planejador_saude",
        model=MODEL_ID, # Usando o modelo definido no setup inicial
        instruction=f"""
        Você é um planejador de conteúdo especializado em comunicação na área da saúde, focado em posts para conectar
        profissionais e pacientes no Instagram. Com base nas informações buscadas sobre o tópico de saúde: "{topico_saude}",
        sua tarefa é analisar essas informações, usar a ferramenta google_search para aprofundar nos detalhes se necessário,
        e criar um plano detalhado para um post engajador, considerando os seguintes pontos:

        1.  **Tema Central:** Escolha o lançamento ou tendência mais relevante das informações buscadas para ser o tema principal do post.
        2.  **Importância para Pacientes:** Identifique e liste os pontos chave dessa informação que são mais relevantes e impactantes para pacientes. Explique o "porquê isso importa" na linguagem de um paciente.
        3.  **Importância para Profissionais:** Identifique e liste os pontos chave dessa informação que são mais relevantes e impactantes para profissionais de saúde. Explique o "porquê isso importa" na linguagem de um profissional.
        4.  **Tópicos a Abordar no Post:** Com base nos pontos relevantes para ambos os públicos, liste os tópicos específicos que o redator deve incluir no post. Seja claro e objetivo.
        5.  **Sugestões de Formato/Abordagem (Instagram):** Dê sugestões criativas sobre como apresentar essa informação no Instagram para maximizar o engajamento (ex: focar em um dado impactante, usar uma analogia simples, sugerir uma pergunta para o público, etc.).

        O plano deve ser conciso, organizado em seções claras (Tema Central, Importância para Pacientes, Importância para Profissionais, Tópicos a Abordar, Sugestões de Formato) e servir como um guia completo para o Agente Redator.

        Informações Buscadas:
        {informacoes_buscadas}
        """,
        description="Agente que planeja posts sobre saúde, focado em clientes e profissionais",
        tools=[google_search]
    )
    entrada_do_agente_planejador = f"Tópico de saúde: {topico_saude}\nInformações Buscadas: {informacoes_buscadas}"
    plano_do_post_saude = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post_saude

### Exemplo de Execução do Agente Planejador de Conteúdo para Conexão Saúde

Agora, vamos ver o Agente Planejador em ação, utilizando as informações que o Agente Buscador encontrou. Ele analisará as informações e gerará um plano detalhado para o post de saúde, focando em como conectar pacientes e profissionais.

In [39]:
# Exemplo de execução do Agente Planejador de Conteúdo para Conexão Saúde
# Certifique-se de que 'informacoes_para_planejar' contém a saída do Agente Buscador
if 'informacoes_para_planejar' in locals() and informacoes_para_planejar:
    print(f"\n🧠 Planejando post sobre {topico_para_planejar} com base nas informações buscadas...")

    # Chama o agente planejador de saúde
    plano_do_post_saude = agente_planejador_saude(topico_para_planejar, informacoes_para_planejar)

    print("\n--- 📝 Resultado do Agente 2 (Planejador de Conteúdo para Conexão Saúde) ---\n")
    display(to_markdown(plano_do_post_saude))
    print("--------------------------------------------------------------")

# Para os próximos exemplos, armazenamos o plano do post
if 'plano_do_post_saude' in locals() and plano_do_post_saude:
    # Variável para ser usada pelos próximos exemplos
    plano_para_redator = plano_do_post_saude
    topico_para_redator = topico_para_planejar # Mantém o tópico para o redator
    print("\n💾 Plano do post armazenado para o próximo agente.")
else:
     print("\n❌ Nenhum plano gerado. Os próximos exemplos podem não funcionar corretamente sem a execução bem-sucedida do Agente Planejador.")


🧠 Planejando post sobre Saúde Mental com base nas informações buscadas...

--- 📝 Resultado do Agente 2 (Planejador de Conteúdo para Conexão Saúde) ---



> Perfeito! Com as informações sobre as tendências em Saúde Mental, posso criar um plano de conteúdo detalhado para o Instagram.
> 
> 
> ## Plano de Conteúdo para Instagram - Saúde Mental e Tecnologia
> 
> **Tema Central:** A Revolução Tecnológica na Saúde Mental: Telemedicina, IA, VR e Wearables
> 
> **Importância para Pacientes:**
> 
> *   **Acessibilidade Ampliada:** A telemedicina e outras tecnologias digitais estão tornando o cuidado com a saúde mental mais acessível, especialmente para quem mora em áreas remotas, tem dificuldades de locomoção ou horários inflexíveis. *Por que isso importa?* Porque agora você pode ter acesso a um psicólogo ou psiquiatra sem sair de casa, no horário que for mais conveniente para você.
> *   **Suporte Personalizado:** A IA e os wearables podem oferecer suporte individualizado, monitorando seu estado emocional e fornecendo feedback em tempo real. *Por que isso importa?* Porque você terá um "assistente" que te ajuda a entender suas emoções e a lidar com elas de forma mais eficaz, no dia a dia.
> *   **Tratamentos Inovadores:** A realidade virtual está abrindo novas possibilidades de tratamento para fobias, ansiedade e traumas. *Por que isso importa?* Porque você pode enfrentar seus medos em um ambiente seguro e controlado, com o apoio de um profissional, sem precisar se expor a situações reais que podem ser muito difíceis.
> 
> **Importância para Profissionais:**
> 
> *   **Ampliação do Alcance:** A telemedicina permite atender pacientes em áreas geográficas mais amplas e oferecer horários mais flexíveis, otimizando o tempo e expandindo a prática. *Por que isso importa?* Porque você pode ajudar mais pessoas e ter mais flexibilidade na sua rotina profissional.
> *   **Ferramentas de Diagnóstico Avançadas:** A IA pode auxiliar no diagnóstico precoce e no monitoramento de pacientes, permitindo intervenções mais rápidas e eficazes. *Por que isso importa?* Porque você terá acesso a dados e análises que podem te ajudar a tomar decisões mais informadas e a oferecer um tratamento mais preciso.
> *   **Terapias Inovadoras:** A realidade virtual oferece novas abordagens terapêuticas para tratar diversos transtornos mentais, abrindo novas áreas de atuação e pesquisa. *Por que isso importa?* Porque você pode oferecer tratamentos mais eficazes e personalizados, utilizando tecnologias de ponta.
> *   **Dados e Insights:** Wearables e aplicativos geram dados valiosos sobre o bem-estar emocional dos pacientes, permitindo um acompanhamento mais preciso e personalizado. *Por que isso importa?* Porque você pode monitorar o progresso dos seus pacientes de forma mais objetiva e ajustar o tratamento conforme necessário.
> 
> **Tópicos a Abordar no Post:**
> 
> 1.  **O que é a "Revolução Tecnológica" na Saúde Mental:** Explicar brevemente o que é telemedicina, IA, VR e wearables e como estão sendo usados na área da saúde mental.
> 2.  **Acessibilidade e Conveniência:** Destacar como a telemedicina facilita o acesso ao tratamento, especialmente para pessoas em áreas remotas ou com horários difíceis. (Mencionar dados estatísticos sobre o aumento do uso da telemedicina).
> 3.  **IA e Suporte Personalizado:** Explicar como a inteligência artificial pode monitorar sintomas de ansiedade e depressão e oferecer suporte personalizado.
> 4.  **Realidade Virtual para Tratamento:** Detalhar como a realidade virtual está sendo usada para tratar fobias, ansiedade e traumas, permitindo que os pacientes enfrentem seus medos em um ambiente seguro. (Mencionar exemplos de transtornos que podem ser tratados com VR, como ansiedade social, TEPT e TOC).
> 5.  **Wearables e Monitoramento Emocional:** Mostrar como dispositivos vestíveis podem monitorar sinais vitais e oferecer feedback em tempo real sobre o estado emocional dos usuários.
> 6.  **Programas de Apoio Psicológico nas Empresas:** Informar sobre a importância de programas de apoio psicológico nas empresas e como eles podem melhorar o bem-estar dos funcionários. (Mencionar benefícios como a redução do estresse, aumento da produtividade e melhora do clima organizacional).
> 7.  **O Futuro da Saúde Mental:** Fazer uma reflexão sobre como a tecnologia continuará transformando a área da saúde mental e como isso pode impactar positivamente a vida de pacientes e profissionais.
> 
> **Sugestões de Formato/Abordagem (Instagram):**
> 
> *   **Carrossel:** Crie um carrossel com cada tópico principal em um slide separado, usando imagens e gráficos atraentes.
> *   **Vídeo Curto:** Faça um vídeo curto explicando como a tecnologia está revolucionando a saúde mental, com depoimentos de pacientes e profissionais.
> *   **Infográfico:** Crie um infográfico com dados estatísticos sobre o uso da telemedicina e outras tecnologias na área da saúde mental.
> *   **Perguntas Interativas:** Faça perguntas interativas nos stories, como "Você já experimentou telemedicina?" ou "Qual tecnologia você acha que terá o maior impacto na saúde mental?".
> *   **Analogia Simples:** Use uma analogia simples para explicar como a realidade virtual funciona, como "Imagine que você pode enfrentar seus medos em um videogame, com a ajuda de um terapeuta".
> *   **Dado Impactante:** Comece o post com um dado impactante sobre a prevalência de transtornos mentais e como a tecnologia pode ajudar a combatê-los. (Ex: "40% dos pacientes corporativos diagnosticados com algum transtorno emocional").
> *   **Colaboração:** Faça uma colaboração com um profissional de saúde mental para criar um conteúdo informativo e confiável.
> *   **Hashtags:** Utilize hashtags relevantes como #saudemental #telemedicina #inteligenciaartificial #realidadevirtual #bemestar #tecnologia #psicologia #psiquiatria #inovacaoemsaude.
> *    **Linguagem:** Utilize uma linguagem acessível e empática, evitando jargões técnicos. Foque em como essas tecnologias podem melhorar a vida das pessoas.
> 
> Este plano de conteúdo deve fornecer uma base sólida para criar um post engajador e informativo sobre a revolução tecnológica na saúde mental. O Agente Redator pode usar essas informações para criar um conteúdo atraente e relevante para o público do Instagram.
> 


--------------------------------------------------------------

💾 Plano do post armazenado para o próximo agente.


## Agente 3: Redator de Posts Engajadores de Saúde

Com o plano detalhado em mãos, o Agente Redator de Posts Engajadores de Saúde transforma o plano em um rascunho de post para o Instagram. Ele utiliza o plano gerado pelo Agente Planejador e aplica as diretrizes de linguagem e tom adequados para engajar o público de saúde, considerando as nuances de comunicação com pacientes e profissionais.

In [40]:
# Definição do Agente Redator de Posts Engajadores de Saúde
def agente_redator_saude(topico_saude: str, plano_de_post_saude: str) -> str:
    """
    Agente que redige um post de Instagram sobre saúde, usando linguagem acessível e engajadora,
    considerando o público de pacientes e profissionais com base em um plano.

    Args:
        topico_saude: O tópico de saúde principal do conteúdo.
        plano_de_post_saude: O plano detalhado gerado pelo Agente Planejador.

    Returns:
        Uma string contendo o rascunho do post para Instagram.
    """
    redator = Agent(
        name="agente_redator_saude",
        model=MODEL_ID, # Usando o modelo definido no setup inicial
        instruction=f"""
            Você é um Redator Criativo especializado em posts para redes sociais na área da saúde, com foco no Instagram.
            Você escreve para um público que inclui tanto pacientes quanto profissionais de saúde.
            Utilize o plano de post fornecido sobre o tópico de saúde ({topico_saude}) e os pontos mais relevantes
            para escrever um rascunho de post para Instagram.

            Diretrizes para a escrita do post:
            - Baseie-se **inteiramente** no plano fornecido. Não adicione informações novas que não estejam no plano.
            - Adote uma linguagem clara, acessível e evite jargões médicos complexos ao se dirigir a pacientes.
            - Destaque o **impacto prático** das informações, tanto para a vida dos pacientes quanto para a rotina ou prática clínica dos profissionais.
            - O tom deve ser informativo, engajador, empático e inspirar confiança. Evite sensacionalismo ou promessas exageradas.
            - Utilize emojis relevantes para a área da saúde, de forma equilibrada.
            - Crie uma introdução cativante que chame a atenção do público no feed.
            - Inclua uma chamada para ação (Call to Action) no final, incentivando a interação (ex: "Deixe sua pergunta nos comentários!", "Compartilhe com quem precisa saber!").
            - Inclua 3 a 5 hashtags relevantes no final. Mescle hashtags populares de saúde (ex: #saude #bemestar) com hashtags mais específicas do tema (ex: #vacinadengue #tecnologiamedica).

            Plano de Post:
            {plano_de_post_saude}
            """,
        description="Agente redator de posts de saúde engajadores para Instagram",
    )
    entrada_do_agente_redator = f"Tópico de saúde: {topico_saude}\nPlano de post de saúde: {plano_de_post_saude}"
    rascunho_saude = call_agent(redator, entrada_do_agente_redator)
    return rascunho_saude

### Exemplo de Execução do Agente Redator de Posts Engajadores de Saúde

Agora, vamos ver o Agente Redator em ação, utilizando o plano gerado pelo Agente Planejador. Ele transformará esse plano em um rascunho de post para o Instagram, aplicando as diretrizes de linguagem e tom para o público de saúde.

In [41]:
# Exemplo de execução do Agente Redator de Posts Engajadores de Saúde
# Certifique-se de que 'plano_para_redator' e 'topico_para_redator' estão definidos
if 'plano_para_redator' in locals() and plano_para_redator and 'topico_para_redator' in locals():
    print(f"\n✍️ Redigindo post sobre {topico_para_redator} com base no plano...")

    # Chama o agente redator de saúde
    rascunho_de_post_saude = agente_redator_saude(topico_para_redator, plano_para_redator)

    print("\n--- 📝 Resultado do Agente 3 (Redator de Posts Engajadores de Saúde) ---\n")
    display(to_markdown(rascunho_de_post_saude))
    print("--------------------------------------------------------------")

# Para os próximos exemplos, armazenamos o rascunho do post
if 'rascunho_de_post_saude' in locals() and rascunho_de_post_saude:
    # Variável para ser usada pelo próximo exemplo
    rascunho_para_revisor = rascunho_de_post_saude
    topico_para_revisor = topico_para_redator # Mantém o tópico para o revisor
    print("\n💾 Rascunho do post armazenado para o próximo agente.")
else:
     print("\n❌ Nenhum rascunho gerado. Os próximos exemplos podem não funcionar corretamente sem a execução bem-sucedida do Agente Redator.")


✍️ Redigindo post sobre Saúde Mental com base no plano...

--- 📝 Resultado do Agente 3 (Redator de Posts Engajadores de Saúde) ---



> 🌟 **Atenção: A saúde mental está evoluindo!** 🌟
> 
> Você sabia que a tecnologia está transformando a forma como cuidamos da nossa mente? 🤯 A telemedicina, a inteligência artificial (IA), a realidade virtual (VR) e os wearables (dispositivos vestíveis) estão revolucionando a saúde mental, tanto para pacientes quanto para profissionais!
> 
> 🤔 **O que é essa "revolução tecnológica"?**
> São novas ferramentas que estão sendo usadas para tornar o cuidado com a saúde mental mais acessível, personalizado e eficaz.
> 
> **Para VOCÊ, paciente:**
> ✅ **Acessibilidade:** A telemedicina te conecta com psicólogos e psiquiatras de onde estiver, economizando tempo e deslocamento.
> 🤖 **Suporte Personalizado:** A IA e os wearables monitoram suas emoções e te dão um "empurrãozinho" quando você mais precisa.
> ✨ **Tratamentos Inovadores:** Já pensou em enfrentar seus medos em um ambiente virtual seguro? A VR está tornando isso possível!
> 
> **Para VOCÊ, profissional de saúde:**
> 🚀 **Amplie seu alcance:** Atenda mais pacientes com a telemedicina e otimize sua agenda.
> 🧠 **Diagnósticos precisos:** A IA te ajuda a identificar problemas precocemente e monitorar seus pacientes de perto.
> 💡 **Terapias inovadoras:** A VR abre um mundo de possibilidades para tratar ansiedade, fobias e traumas.
> 
> 🏢 **Empresas:** Sabiam que o apoio psicológico no trabalho reduz o estresse e aumenta a produtividade?
> 
> O futuro da saúde mental já chegou! 🚀 Quer saber mais sobre como a tecnologia pode te ajudar? Deixe suas dúvidas nos comentários! 👇
> 
> #saudemental #telemedicina #inteligenciaartificial #realidadevirtual #bemestar #tecnologia #psicologia #inovacaoemsaude
> 


--------------------------------------------------------------

💾 Rascunho do post armazenado para o próximo agente.


## Agente 4: Revisor de Qualidade em Saúde

O último agente em nosso pipeline é o Revisor de Qualidade em Saúde. Sua função é crucial para garantir a precisão, clareza e adequação do conteúdo gerado, especialmente em um campo tão sensível quanto a saúde. Ele avalia o rascunho do post com base em critérios rigorosos e pode aprovar o post ou sugerir melhorias.

In [42]:
# Definição do Agente Revisor de Qualidade em Saúde
def agente_revisor_saude(topico_saude: str, rascunho_gerado_saude: str) -> str:
    """
    Agente que revisa posts de saúde, focando em precisão, clareza, tom e adequação ao público.

    Args:
        topico_saude: O tópico de saúde principal do conteúdo.
        rascunho_gerado_saude: O rascunho do post gerado pelo Agente Redator.

    Returns:
        Uma string contendo a revisão do post (aprovação ou sugestões de melhoria).
    """
    revisor = Agent(
        name="agente_revisor_saude",
        model=MODEL_ID, # Usando o modelo definido no setup inicial
        instruction=f"""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais na área da saúde, com foco no Instagram.
            Seu objetivo é garantir que o rascunho de post abaixo sobre o tópico de saúde ({topico_saude}) seja preciso, claro, conciso, com o tom adequado e relevante para o público de pacientes e profissionais de saúde.

            Critérios de revisão:
            - **Precisão da informação:** Verifique se as informações médicas apresentadas são precisas e não enganosas.
            - **Clareza e Acessibilidade:** O texto é fácil de entender para pacientes leigos? Evita jargões desnecessários?
            - **Relevância para o público:** O post aborda o impacto prático para pacientes e profissionais?
            - **Tom e Linguagem:** O tom é informativo, engajador e empático? A linguagem é apropriada para o Instagram e o tema de saúde?
            - **Concisão:** O post é direto ao ponto, sem informações redundantes?
            - **Hashtags:** As hashtags são relevantes e bem escolhidas?
            - **Conformidade (Opcional - mencione a complexidade):** Embora não seja o foco principal, o post evita linguagem que possa ser interpretada como aconselhamento médico direto?

            Se o rascunho estiver ótimo e atender a todos os critérios, responda apenas '✅ O rascunho está ótimo e pronto para publicar na área da saúde! 🩺'.
            Caso haja problemas, aponte-os de forma clara e sugira melhorias específicas, explicando o porquê da sugestão em relação aos critérios de revisão para a área da saúde. Use emojis para destacar os pontos de melhoria (ex: ⚠️, ✏️).

            Rascunho do Post para Revisão:
            {rascunho_gerado_saude}
            """,
        description="Agente revisor de post de saúde para redes sociais.",
    )
    entrada_do_agente_revisor = f"Tópico de saúde: {topico_saude}\nRascunho de post de saúde: {rascunho_gerado_saude}"
    texto_revisado_saude = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado_saude

### Exemplo de Execução do Agente Revisor de Qualidade em Saúde

O último agente em nosso pipeline, o Revisor de Qualidade, avalia o rascunho final. Ele verifica a precisão, clareza e adequação para o público de saúde. O resultado será uma aprovação ou sugestões de melhoria para o rascunho gerado pelo Agente Redator.

In [43]:
# Exemplo de execução do Agente Revisor de Qualidade em Saúde
# Certifique-se de que 'rascunho_para_revisor' e 'topico_para_revisor' estão definidos
if 'rascunho_para_revisor' in locals() and rascunho_para_revisor and 'topico_para_revisor' in locals():
    print(f"\n✨ Revisando o post sobre {topico_para_revisor} para qualidade e precisão...")

    # Chama o agente revisor de saúde
    post_final_saude = agente_revisor_saude(topico_para_revisor, rascunho_para_revisor)

    print("\n--- ✅ Resultado do Agente 4 (Revisor de Qualidade em Saúde) ---\n")
    display(to_markdown(post_final_saude))
    print("--------------------------------------------------------------")

else:
     print("\n❌ Nenhum rascunho disponível para revisão. Os próximos exemplos podem não funcionar corretamente sem a execução bem-sucedida do Agente Redator.")


✨ Revisando o post sobre Saúde Mental para qualidade e precisão...

--- ✅ Resultado do Agente 4 (Revisor de Qualidade em Saúde) ---



> ✅ O rascunho está ótimo e pronto para publicar na área da saúde! 🩺


--------------------------------------------------------------


## Próximos Passos e Potenciais Melhorias

Este projeto demonstra o potencial da IA na criação de conteúdo para a área da saúde, focando na conexão entre pacientes e profissionais. Há diversas possibilidades de expansão e melhoria para tornar o sistema ainda mais completo e útil:

*   **Personalização Avançada:** Implementar funcionalidades para personalizar o conteúdo de forma mais granular com base no perfil específico de pacientes (idade, condição de saúde, interesses) ou profissionais de saúde (especialidade, área de atuação).
*   **Integração com Outras Fontes de Dados:** Conectar o sistema a bases de dados de pesquisa científica (como PubMed, Scielo), diretrizes clínicas atualizadas, notícias de agências de saúde (como Anvisa, FDA) e outras fontes relevantes para enriquecer a informação buscada.
*   **Novos Agentes Especializados:** Desenvolver agentes adicionais para tarefas específicas, como um agente para gerar legendas para vídeos educativos, um agente para criar infográficos simples a partir de dados de saúde, ou um agente especializado em conteúdo para campanhas de saúde pública.
*   **Loop de Feedback:** Incorporar um mecanismo para coletar feedback sobre o conteúdo gerado (por exemplo, de profissionais de saúde revisores ou até mesmo métricas de engajamento em redes sociais) e usar esse feedback para aprimorar as instruções e o desempenho dos agentes ao longo do tempo.
*   **Suporte a Outros Formatos:** Expandir a capacidade de gerar conteúdo para outros formatos além de posts de Instagram, como artigos de blog, roteiros para podcasts curtos, ou conteúdo para newsletters informativas.
*   **Interface de Usuário:** Desenvolver uma interface de usuário mais amigável para facilitar a interação com o sistema, permitindo que os usuários insiram tópicos, visualizem os resultados de cada agente e editem o conteúdo final antes da publicação.